---
# MNIST em uma CNN usando Keras
---

From: https://github.com/fchollet/keras-resources

## Treina uma rede convolucional simples com a base de dados do MNIST

Reportado pelo autor:
* Gets to 99.25% test accuracy after 12 epochs (there is still a lot of margin for parameter tuning).
* 16 seconds per epoch on a GRID K520 GPU.

Resultados obtidos:
* 98.94% de acurária no teste, depois de 12 épocas.
* 30 segundos por época em uma GPU GRID K520 (dentro do Docker).

## Preparação do sistema para execução

Verificar a GPU atual. Este comando mostra que a GRID K520 é a GPU0. 

In [1]:
!nvidia-smi

Wed Jan 25 18:16:25 2017       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 367.57                 Driver Version: 367.57                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GRID K520           Off  | 0000:00:03.0     Off |                  N/A |
| N/A   28C    P8    17W / 125W |      0MiB /  4036MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

## Primeiros imports

É preciso configurar o Theano para utilizar a GPU, o que também requere usar float32, antes de importá-lo. Uma vez que o mesmo já tenha sido importado não é possível alterar isto sem reiniciar o Kernel.

Por padrão o Theano usa a CPU para processamento.

In [2]:
from __future__ import print_function
import numpy as np
np.random.seed(1337)  # for reproducibility
import os
os.environ['THEANO_FLAGS'] = "device=gpu0,floatX=float32"    

import theano

Using gpu device 0: GRID K520 (CNMeM is disabled, CuDNN not available)


## Imports adicionais do Keras

In [3]:
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils
from keras import backend as K

Using Theano backend.


## Configurações básicas da rede e das operações a serem executadas

In [4]:
batch_size = 128
nb_classes = 10
nb_epoch = 12

# input image dimensions
img_rows, img_cols = 28, 28
# number of convolutional filters to use
nb_filters = 32
# size of pooling area for max pooling
pool_size = (2, 2)
# convolution kernel size
kernel_size = (3, 3)

## Download da base de dados do MNIST e divisão em conjuntos de treino e teste

In [5]:
# the data, shuffled and split between train and test sets
(X_train, y_train), (X_test, y_test) = mnist.load_data()

if K.image_dim_ordering() == 'th':
    X_train = X_train.reshape(X_train.shape[0], 1, img_rows, img_cols)
    X_test = X_test.reshape(X_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
    X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

15302656/15296311 [==============================] - 1s     


In [6]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

X_train shape: (60000, 1, 28, 28)
60000 train samples
10000 test samples


In [7]:
# convert class vectors to binary class matrices
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

## Criação do modelo da rede neural convolucional

In [8]:
model = Sequential()

model.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1],
                        border_mode='valid',
                        input_shape=input_shape))
model.add(Activation('relu'))
model.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=pool_size))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adadelta',
              metrics=['accuracy'])

## Treinando o modelo com o conjunto de treino separado

In [9]:
model.fit(X_train, Y_train, batch_size=batch_size, nb_epoch=nb_epoch,
          verbose=1, validation_data=(X_test, Y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/12
60000/60000 [==============================] - 30s - loss: 0.3815 - acc: 0.8828 - val_loss: 0.1019 - val_acc: 0.9680
Epoch 2/12
60000/60000 [==============================] - 30s - loss: 0.1520 - acc: 0.9557 - val_loss: 0.0684 - val_acc: 0.9783
Epoch 3/12
60000/60000 [==============================] - 30s - loss: 0.1172 - acc: 0.9650 - val_loss: 0.0555 - val_acc: 0.9817
Epoch 4/12
60000/60000 [==============================] - 29s - loss: 0.0998 - acc: 0.9709 - val_loss: 0.0480 - val_acc: 0.9845
Epoch 5/12
60000/60000 [==============================] - 29s - loss: 0.0870 - acc: 0.9752 - val_loss: 0.0452 - val_acc: 0.9852
Epoch 6/12
60000/60000 [==============================] - 29s - loss: 0.0781 - acc: 0.9767 - val_loss: 0.0432 - val_acc: 0.9851
Epoch 7/12
60000/60000 [==============================] - 29s - loss: 0.0676 - acc: 0.9797 - val_loss: 0.0374 - val_acc: 0.9873
Epoch 8/12
60000/60000 [==============================

## Testando a acurácia do modelo com o conjunto de teste separado

In [10]:
score = model.evaluate(X_test, Y_test, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])

Test score: 0.0307071016752
Test accuracy: 0.9894
